# Spike Gadgets Ephys and Video Syncing

In [154]:
import os
import re
from sys import argv
from collections import defaultdict
import random
from random import randrange

In [115]:
import numpy as np
import cv2

## File to Numpy Array Functions from Trodes

In [3]:
# Main function
def readTrodesExtractedDataFile(filename):
    with open(filename, 'rb') as f:
        # Check if first line is start of settings block
        # TODO: Uncomment if it works
        if f.readline().decode('ascii').strip() != '<Start settings>':
#         if f.readline().strip() != '<Start settings>':

            raise Exception("Settings format not supported")
        fields = True
        fieldsText = {}
        for line in f:
            # Read through block of settings
            if(fields):
                # TODO: Uncomment if it works
                line = line.decode('ascii').strip()
#                 line = line.strip()

                # filling in fields dict
                if line != '<End settings>':
                    vals = line.split(': ')
                    fieldsText.update({vals[0].lower(): vals[1]})
                # End of settings block, signal end of fields
                else:
                    fields = False
                    dt = parseFields(fieldsText['fields'])
                    fieldsText['data'] = np.zeros([1], dtype = dt)
                    break
        # Reads rest of file at once, using dtype format generated by parseFields()
        dt = parseFields(fieldsText['fields'])
        data = np.fromfile(f, dt)
        fieldsText.update({'data': data})
        return fieldsText




In [4]:
# Parses last fields parameter (<time uint32><...>) as a single string
# Assumes it is formatted as <name number * type> or <name type>
# Returns: np.dtype
def parseFields(fieldstr):
    # Returns np.dtype from field string
    sep = re.split('\s', re.sub(r"\>\<|\>|\<", ' ', fieldstr).strip())
    # print(sep)
    typearr = []
    # Every two elmts is fieldname followed by datatype
    for i in range(0,sep.__len__(), 2):
        fieldname = sep[i]
        repeats = 1
        ftype = 'uint32'
        # Finds if a <num>* is included in datatype
        if sep[i+1].__contains__('*'):
            temptypes = re.split('\*', sep[i+1])
            # Results in the correct assignment, whether str is num*dtype or dtype*num
            ftype = temptypes[temptypes[0].isdigit()]
            repeats = int(temptypes[temptypes[1].isdigit()])
        else:
            ftype = sep[i+1]
        try:
            fieldtype = getattr(np, ftype)
        except AttributeError:
            print(ftype + " is not a valid field type.\n")
            exit(1)
        else:
            typearr.append((str(fieldname), fieldtype, repeats))
    return np.dtype(typearr)

## Extracting the data and the metadata from the Recording folder

In [5]:
recording_directory = "../../video_testing/minilogger_with_mcu_20220830_141749.rec/"

- Creating a dictionary that has the directory as the key and a dictionary that has the file name as the key and the 

In [6]:
file_to_metadata = defaultdict(dict)
for root, subdirs, files in os.walk(recording_directory):
#     print("Root: {}".format(root))
#     print("Subdirs: {}".format(subdirs))
#     print("Files: {}".format(files))
    for file_name in files:
        try:
            file_path = os.path.join(root, file_name)
            trodes_recording = readTrodesExtractedDataFile(file_path)
            directory_name = os.path.basename(root)
            file_to_metadata[directory_name][file_name] = {}            
            file_to_metadata[directory_name][file_name]["file_path"] = file_path
            file_to_metadata[directory_name][file_name]["fields_text"] = trodes_recording
#             print("File Path: {}".format(os.path.join(root, file_name)))
            for key, item in trodes_recording.items():
                pass
#                 print("Key: {}".format(key))
#                 print("Item: {}".format(item))
        except:
#             pass
            print("File does not work: {}".format(file_name))
        
    
    
    

File does not work: logger_raw.dat
File does not work: minilogger_with_mcu_20220830_141749.1.h264
File does not work: minilogger_with_mcu_20220830_141749.1.mp4
File does not work: minilogger_with_mcu_20220830_141749.1.trackgeometry
File does not work: minilogger_with_mcu_20220830_141749.1.videoTimeStamps
File does not work: minilogger_with_mcu_20220830_141749.rec
File does not work: minilogger_with_mcu_20220830_141749_merged.rec
File does not work: minilogger_with_mcu_20220830_141749_merged.trodesconf
File does not work: temp_08302022_1424.trodesconf


/tmp/ipykernel_1651/1163065242.py:29: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  return np.dtype(typearr)


File does not work: minilogger_with_mcu_20220830_141749_merged.group0.dat
File does not work: minilogger_with_mcu_20220830_141749_merged.group0.mda
File does not work: minilogger_with_mcu_20220830_141749_merged.timestamps.mda


- All the folders that were used in 

In [7]:
file_to_metadata.keys()

dict_keys(['', 'minilogger_with_mcu_20220830_141749_merged.analog', 'minilogger_with_mcu_20220830_141749_merged.DIO', 'minilogger_with_mcu_20220830_141749_merged.kilosort', 'minilogger_with_mcu_20220830_141749_merged.LFP', 'minilogger_with_mcu_20220830_141749_merged.mountainsort', 'minilogger_with_mcu_20220830_141749_merged.raw', 'minilogger_with_mcu_20220830_141749_merged.spikeband', 'minilogger_with_mcu_20220830_141749_merged.spikes', 'minilogger_with_mcu_20220830_141749_merged.stimulation'])

In [8]:
for key, value in file_to_metadata.items():
    print("Directory: {}".format(key))
    for file_name, trodes_dict in value.items():
        print("\tFile Name: {}".format(file_name))
        for data_type, data_value in trodes_dict["fields_text"].items():
            print("\n \t \t Trodes Data Type: {}".format(data_type))
            print("\t \t Python Type: {}".format(type(data_value)))
            print("\t \t Data Value: {}".format(data_value))

Directory: 
	File Name: minilogger_with_mcu_20220830_141749.1.videoTimeStamps.cameraHWSync

 	 	 Trodes Data Type: clock rate
	 	 Python Type: <class 'str'>
	 	 Data Value: 30000

 	 	 Trodes Data Type: fields
	 	 Python Type: <class 'str'>
	 	 Data Value: <PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>

 	 	 Trodes Data Type: data
	 	 Python Type: <class 'numpy.ndarray'>
	 	 Data Value: [( 558504, 1945, 0) ( 558854, 1946, 0) ( 559890, 1947, 0) ...
 (2691357, 5126, 0) (2691357, 5127, 0) (2692743, 5128, 0)]
Directory: minilogger_with_mcu_20220830_141749_merged.analog
	File Name: minilogger_with_mcu_20220830_141749_merged.analog_Controller_Ain1.dat

 	 	 Trodes Data Type: description
	 	 Python Type: <class 'str'>
	 	 Data Value: One analog IO channel. Continuous sampling.

 	 	 Trodes Data Type: byte_order
	 	 Python Type: <class 'str'>
	 	 Data Value: little endian

 	 	 Trodes Data Type: original_file
	 	 Python Type: <class 'str'>
	 	 Data Value: minilogger_with_mcu_20

- Array of Voltages for each channel

In [9]:
voltage_value_array = file_to_metadata["minilogger_with_mcu_20220830_141749_merged.raw"]["minilogger_with_mcu_20220830_141749_merged.raw_group0.dat"]["fields_text"]["data"]

In [10]:
voltage_value_array.shape

(2134238,)

In [11]:
voltage_value_array[:5]

array([([  5,   5, -22,  25, -15, -16,   4, -10, -14,  17,   8,  -7,   7, -11,  -4,  24,  13, -28,  16,   9,  15, -37,  10, -22,  -3,  10,  -2, -12,  -1, -24, -28,   0],),
       ([ 15,  13,  -3,  30, -15,  -1, -11,  13,  -2, -10,  -2,  13,   0,  -5, -15,   9,  12, -24,  17,  -1,  26, -17, -11, -10,   3,  -6,  -8,  -1,  -6,  -6, -10,  10],),
       ([ -6,  12,  -8,   8,  -6,  -1,   4,  -6,   5,   4,  -3,   7,  17,   6,  -2,  21,  12,  -4,   2, -12,  30, -24,   9, -26,  -5,  19, -10,   7, -22, -15,   8,   2],),
       ([  3,   6,  -4,  23, -16, -12, -22,   5,   0,   6,   8, -11,  12, -10,  -3,  25,  19,  -8,  16, -11,  11, -31,  17, -21,  -1,  12,  11,  17,  -7, -13,  -9,   4],),
       ([ -1,  -1,  -4,  24,  -4, -15, -18,   5, -28, -16,  -5,   7,  11,  -1, -18,  29,  11,  -9,  -6, -11,  -6, -28, -15, -37,   3,   0,  18,   5, -14,  -1, -33, -11],)],
      dtype=[('voltage', '<i2', (32,))])

- Voltage Time Stamps

    - u4 and i2 explanation: https://www.geeksforgeeks.org/data-type-object-dtype-numpy-python/

In [12]:
voltage_timestamp_array = file_to_metadata["minilogger_with_mcu_20220830_141749_merged.raw"]["minilogger_with_mcu_20220830_141749_merged.timestamps.dat"]["fields_text"]["data"]

In [13]:
voltage_timestamp_array[:5]

array([(558506,), (558507,), (558508,), (558509,), (558510,)],
      dtype=[('time', '<u4')])

- Converting the array to integers to be able to do calculations

In [14]:
voltage_timestamp_array = voltage_timestamp_array.astype(int)

In [15]:
voltage_timestamp_array[:5]

array([558506, 558507, 558508, 558509, 558510])

In [16]:
voltage_timestamp_array[-5:]

array([2692739, 2692740, 2692741, 2692742, 2692743])

In [17]:
voltage_timestamp_array.shape

(2134238,)

- Video Time Stamps

In [18]:
video_time_stamp_dict = file_to_metadata[""]["minilogger_with_mcu_20220830_141749.1.videoTimeStamps.cameraHWSync"]["fields_text"]

In [19]:
video_time_stamp_dict

{'clock rate': '30000',
 'fields': '<PosTimestamp uint32><HWframeCount uint32><HWTimestamp uint64>',
 'data': array([( 558504, 1945, 0), ( 558854, 1946, 0), ( 559890, 1947, 0), ...,
        (2691357, 5126, 0), (2691357, 5127, 0), (2692743, 5128, 0)],
       dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])}

In [20]:
video_time_stamp_dict["data"]

array([( 558504, 1945, 0), ( 558854, 1946, 0), ( 559890, 1947, 0), ...,
       (2691357, 5126, 0), (2691357, 5127, 0), (2692743, 5128, 0)],
      dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])

In [21]:
video_time_stamp_dict["data"].shape

(3184,)

# Syncing the Ephys with the Video

# From Ephys to Video

- Getting a random recording instance from ephys
    - This will be by getting a random number from 0 to the number of recording instances. This can be used as the index for the ephys recording

In [93]:
random_voltage_index = np.random.choice(voltage_timestamp_array.shape[0], 1, replace=False)  

In [94]:
random_voltage_index

array([1962784])

In [95]:
voltage_value_array[random_voltage_index]

array([([ -7,  17,   9,   9, -40,  17,  -2, -22, -13,  34,  -9,   8,  25, -13,  11, -24, -16,   5, -10, -15,  -5, -30,  22,  -8,  -4,   4,   7, -18,  24,   9,   2,  -6],)],
      dtype=[('voltage', '<i2', (32,))])

- Getting the associated timestamp of the ephys recording

In [97]:
random_voltage_time_stamp = voltage_timestamp_array[random_voltage_index]

In [98]:
random_voltage_time_stamp

array([2521290])

- Getting the time stamps of the video frames

In [99]:
video_time_stamp_array = np.array(video_time_stamp_dict["data"])

In [100]:
video_time_stamp_array

array([( 558504, 1945, 0), ( 558854, 1946, 0), ( 559890, 1947, 0), ...,
       (2691357, 5126, 0), (2691357, 5127, 0), (2692743, 5128, 0)],
      dtype=[('PosTimestamp', '<u4'), ('HWframeCount', '<u4'), ('HWTimestamp', '<u8')])

- Getting only the first number in each tuple

In [104]:
pos_timestamp_array = np.array([x[0] for x in video_time_stamp_array]) 

In [105]:
pos_timestamp_array

array([ 558504,  558854,  559890, ..., 2691357, 2691357, 2692743],
      dtype=uint32)

- Converting to integer to do calculations

In [108]:
pos_timestamp_array = pos_timestamp_array.astype(int)

In [109]:
pos_timestamp_array[:5]

array([558504, 558854, 559890, 561183, 561276])

- Getting the first video time stamp that is greater than the voltage time stamp

In [110]:
current_video_time_stamp = pos_timestamp_array[pos_timestamp_array >= random_voltage_time_stamp][0]

In [111]:
current_video_time_stamp

2522281

- Where this video time stamp is(within the list of video time stamps), would be the frame number that corresponds to the ephys recording instance

In [112]:
current_video_frame = np.argwhere(pos_timestamp_array >= random_voltage_time_stamp)[0]

In [113]:
current_video_frame

array([2929])

In [114]:
current_video_frame[0]

2929

# Reading in video

In [116]:
recording_directory

'../../video_testing/minilogger_with_mcu_20220830_141749.rec/'

In [117]:
video_file = '../../video_testing/minilogger_with_mcu_20220830_141749.rec/minilogger_with_mcu_20220830_141749.1.mp4'

In [118]:
# read video from file
cap = cv2.VideoCapture(video_file)

# Converting from frame to video

- Getting the number of frames per second

In [127]:
fps = cap.get(cv2.CAP_PROP_FPS)

In [128]:
fps

15.004789464686183

- Getting the time in the video that the frame corresponds to
    - Dividing the frame number by the frames per second

In [121]:
current_video_frame[0] / fps

195.2043383809823

- In minutes and seconds

In [132]:
current_video_frame[0] / fps // 60

3.0

In [133]:
current_video_frame[0] / fps % 60

15.204338380982307

## Looking over the Video

In [131]:
Video(video_file)

# From Video to Ephys

- Getting the total number of frames in the video

In [148]:
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

In [149]:
total_frames

3183.0

- Calculating the duration of the video
    - Dividing the total number of frames by the frames per second

In [150]:
video_duration = total_frames / fps

In [151]:
video_duration

212.13226666666665

- Getting a random number that's within the duration of the video

In [159]:
random_video_time = random.uniform(0, video_duration)

In [160]:
random_video_time

162.0212411459788

- Getting the frame number from the video time
    - Multiplying the video time by the fps

In [162]:
random_video_frame = random_video_time * fps

In [163]:
random_video_frame

2431.094612202562

- Getting the video time stamp that's the closest to the frame

In [168]:
random_video_time_stamp = pos_timestamp_array[int(random_video_frame)]

In [169]:
random_video_time_stamp

2188287

- Getting the voltage recording instance that has the closest timestamp to the video frame's

In [172]:
voltage_timestamp_array[voltage_timestamp_array >= random_video_time_stamp][0]

2188287

In [175]:
current_voltage_index = np.argwhere(voltage_timestamp_array >= random_video_time_stamp)[0]

In [176]:
current_voltage_index

array([1629781])

In [177]:
voltage_value_array[current_voltage_index]

array([([  1,  13,  -1,   3,   4,   7,   3,  15,   1,   8,  -6, -13,  15,  -8,   8,  -1,  24, -22,   6,  -5,  -2, -11,   7,  10,  13,  16,  -1,  13, -15,   1,  -7, -16],)],
      dtype=[('voltage', '<i2', (32,))])